In [ ]:
import os
import h5py
import numpy as np
import tensorflow as tf
from skimage.transform import resize
from sklearn.model_selection import train_test_split
tf.enable_eager_execution()
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [ ]:
num_classes     = 9
batch_size      = 64
train_src_days  = 6
train_trg_days  = 0
train_trg_env_days = 2
epochs          = 400
init_lr         = 0.0001
num_features    = 256
gen_activation  = 'selu'
notes           = "resnet_server_conference_labeled"

log_data = "classes-{}_bs-{}_train_src_days-{}_train_trg_days-{}_train_trgenv_days-{}_initlr-{}_num_feat-{}_gen_act-{}_{}".format(num_classes,
                                                                                                                                    batch_size,
                                                                                                                                    train_src_days,
                                                                                                                                    train_trg_days,
                                                                                                                                    train_trg_env_days,
                                                                                                                                    init_lr,
                                                                                                                                    num_features,
                                                                                                                                    gen_activation,
                                                                                                                                    notes)

In [ ]:
def resize_data(data, output_shape=(224, 224)):
  _, height, width, channels = data.shape
  data = data.transpose((1, 2, 3, 0))
  data = resize(data.reshape(height, width, -1), output_shape)
  data = data.reshape(*output_shape, channels, -1)
  data = data.transpose((3, 0, 1, 2))
  return data

#Read data
hf = h5py.File('/home/kjakkala/mmwave/data/source_data.h5', 'r')
X_data = resize_data(np.expand_dims(hf.get('X_data'), axis=-1))
y_data = np.array(hf.get('y_data'))
classes = list(hf.get('classes'))
classes = [n.decode("ascii", "ignore") for n in classes]
hf.close()
print(X_data.shape, y_data.shape, "\n", classes)

#balence dataset to 95 samples per day for each person
X_data_tmp = []
y_data_tmp = []
for day in range(10):
  for idx in range(len(classes)):
    X_data_tmp.extend(X_data[(y_data[:, 0] == idx) & (y_data[:, 1] == day)][:95])
    y_data_tmp.extend(y_data[(y_data[:, 0] == idx) & (y_data[:, 1] == day)][:95])
X_data = np.array(X_data_tmp)
y_data = np.array(y_data_tmp)
del X_data_tmp, y_data_tmp
print(X_data.shape, y_data.shape)

#remove harika's data
X_data = np.delete(X_data, np.where(y_data[:, 0] == 1)[0], 0)
y_data = np.delete(y_data, np.where(y_data[:, 0] == 1)[0], 0)

#update labes to handle 9 classes instead of 10
y_data[y_data[:, 0] >= 2, 0] -= 1
del classes[1]
print(X_data.shape, y_data.shape, "\n", classes)

#split days of data to train and test
X_src = X_data[y_data[:, 1] < train_src_days]
y_src = y_data[y_data[:, 1] < train_src_days, 0]
y_src = np.eye(len(classes))[y_src]
X_train_src, X_test_src, y_train_src, y_test_src = train_test_split(X_src,
                                                                    y_src,
                                                                    stratify=y_src,
                                                                    test_size=0.10,
                                                                    random_state=42)

X_trg = X_data[y_data[:, 1] >= train_src_days]
y_trg = y_data[y_data[:, 1] >= train_src_days]
X_train_trg = X_trg[y_trg[:, 1] < train_src_days+train_trg_days]
y_train_trg = y_trg[y_trg[:, 1] < train_src_days+train_trg_days, 0]
y_train_trg = np.eye(len(classes))[y_train_trg]

X_test_trg = X_data[y_data[:, 1] >= train_src_days+train_trg_days]
y_test_trg = y_data[y_data[:, 1] >= train_src_days+train_trg_days, 0]
y_test_trg = np.eye(len(classes))[y_test_trg]

del X_src, y_src, X_trg, y_trg, X_data, y_data

#standardise dataset
src_mean = np.mean(X_train_src)
X_train_src -= src_mean
src_std  = np.std(X_train_src)
X_train_src /= src_std

X_test_src -= src_mean
X_test_src /= src_std

trg_mean = np.mean(X_train_trg)
X_train_trg -= trg_mean
trg_std  = np.std(X_train_trg)
X_train_trg /= trg_std

X_test_trg -= src_mean
X_test_trg /= src_std

X_train_src = X_train_src.astype(np.float32)
y_train_src = y_train_src.astype(np.uint8)
X_test_src  = X_test_src.astype(np.float32)
y_test_src  = y_test_src.astype(np.uint8)
X_train_trg = X_train_trg.astype(np.float32)
y_train_trg = y_train_trg.astype(np.uint8)
X_test_trg  = X_test_trg.astype(np.float32)
y_test_trg  = y_test_trg.astype(np.uint8)

print(X_train_src.shape, y_train_src.shape,  X_test_src.shape, y_test_src.shape, X_train_trg.shape, y_train_trg.shape, X_test_trg.shape, y_test_trg.shape)

def get_trg_data(fname, src_classes, train_trg_days):
  #Read data
  hf = h5py.File(fname, 'r')
  X_data_trg = resize_data(np.expand_dims(hf.get('X_data'), axis=-1))
  y_data_trg = np.array(hf.get('y_data'))
  trg_classes = list(hf.get('classes'))
  trg_classes = [n.decode("ascii", "ignore") for n in trg_classes]
  hf.close()

  #split days of data to train and test
  X_train_trg = X_data_trg[y_data_trg[:, 1] < train_trg_days]
  y_train_trg = y_data_trg[y_data_trg[:, 1] < train_trg_days, 0]
  y_train_trg = np.array([src_classes.index(trg_classes[y_train_trg[i]]) for i in range(y_train_trg.shape[0])])
  y_train_trg = np.eye(len(src_classes))[y_train_trg]
  y_train_trg = y_train_trg.astype(np.int64)

  X_test_trg = X_data_trg[y_data_trg[:, 1] >= train_trg_days]
  y_test_trg = y_data_trg[y_data_trg[:, 1] >= train_trg_days, 0]
  y_test_trg = np.eye(len(src_classes))[y_test_trg]
  y_test_trg = y_test_trg.astype(np.int64)

  #standardise dataset
  trg_mean     = np.mean(X_train_trg)
  X_train_trg -= trg_mean
  trg_std      = np.std(X_train_trg)
  X_train_trg /= trg_std

  X_test_trg  -= trg_mean
  X_test_trg  /= trg_std

  return X_train_trg.astype(np.float32), y_train_trg.astype(np.uint8), X_test_trg.astype(np.float32), y_test_trg.astype(np.uint8)

X_train_conf,   y_train_conf,   X_test_conf,   y_test_conf   = get_trg_data('/home/kjakkala/mmwave/data/target_conf_data.h5',   classes, train_trg_env_days)
X_train_server, y_train_server, X_test_server, y_test_server = get_trg_data('/home/kjakkala/mmwave/data/target_server_data.h5', classes, train_trg_env_days)
X_data_office,  y_data_office,  _,             _             = get_trg_data('/home/kjakkala/mmwave/data/target_office_data.h5', classes, 3)

print(X_train_conf.shape,   y_train_conf.shape,    X_test_conf.shape,   y_test_conf.shape)
print(X_train_server.shape, y_train_server.shape,  X_test_server.shape, y_test_server.shape)
print(X_data_office.shape, y_data_office.shape)

#get tf.data objects for each set

#Test
conf_test_set = tf.data.Dataset.from_tensor_slices((X_test_conf, y_test_conf))
conf_test_set = conf_test_set.batch(batch_size, drop_remainder=False)
conf_test_set = conf_test_set.prefetch(batch_size)

server_test_set = tf.data.Dataset.from_tensor_slices((X_test_server, y_test_server))
server_test_set = server_test_set.batch(batch_size, drop_remainder=False)
server_test_set = server_test_set.prefetch(batch_size)

office_test_set = tf.data.Dataset.from_tensor_slices((X_data_office, y_data_office))
office_test_set = office_test_set.batch(batch_size, drop_remainder=False)
office_test_set = office_test_set.prefetch(batch_size)

src_test_set = tf.data.Dataset.from_tensor_slices((X_test_src, y_test_src))
src_test_set = src_test_set.batch(batch_size, drop_remainder=False)
src_test_set = src_test_set.prefetch(batch_size)

time_test_set = tf.data.Dataset.from_tensor_slices((X_test_trg, y_test_trg))
time_test_set = time_test_set.batch(batch_size, drop_remainder=False)
time_test_set = time_test_set.prefetch(batch_size)

#Train
src_train_set = tf.data.Dataset.from_tensor_slices((X_train_src, y_train_src))
src_train_set = src_train_set.shuffle(X_train_src.shape[0])
src_train_set = src_train_set.batch(batch_size, drop_remainder=True)
src_train_set = src_train_set.prefetch(batch_size)

server_train_set = tf.data.Dataset.from_tensor_slices((X_train_server, y_train_server))
server_train_set = server_train_set.shuffle(X_train_server.shape[0])
server_train_set = server_train_set.batch(batch_size, drop_remainder=True)
server_train_set = server_train_set.prefetch(batch_size)
server_train_set = server_train_set.repeat(-1)

conf_train_set = tf.data.Dataset.from_tensor_slices((X_train_conf, y_train_conf))
conf_train_set = conf_train_set.shuffle(X_train_conf.shape[0])
conf_train_set = conf_train_set.batch(batch_size, drop_remainder=True)
conf_train_set = conf_train_set.prefetch(batch_size)
conf_train_set = conf_train_set.repeat(-1)

In [ ]:
L2_WEIGHT_DECAY = 1e-4
BATCH_NORM_DECAY = 0.9
BATCH_NORM_EPSILON = 1e-5

class GaussianNoise(tf.keras.layers.Layer):
  def __init__(self, std):
    super(GaussianNoise, self).__init__()
    self.std = std

  def call(self, inputs, training=False):
    eps = tf.random.normal(shape=tf.shape(inputs), mean=0.0, stddev=self.std)
    return tf.where(training, inputs + eps, inputs)

class IdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters, stage, block, activation='relu'):
    self.activation = activation

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    super().__init__(name='stage-' + str(stage) + '_block-' + block)

    filters1, filters2, filters3 = filters
    bn_axis = -1

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1),
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2a')
    self.bn2a = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2a')
    self.act1  = tf.keras.layers.Activation(self.activation)

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size,
                                         padding='same',
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2b')
    self.bn2b = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2b')
    self.act2  = tf.keras.layers.Activation(self.activation)

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1),
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2c')
    self.bn2c = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2c')
    self.act3  = tf.keras.layers.Activation(self.activation)

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = self.act1(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = self.act2(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x = tf.keras.layers.add([x, input_tensor])
    x = self.act3(x)
    return x


"""A block that has a conv layer at shortcut.

Note that from stage 3,
the second conv layer at main path is with strides=(2, 2)
And the shortcut should have strides=(2, 2) as well

Args:
  kernel_size: the kernel size of middle conv layer at main path
  filters: list of integers, the filters of 3 conv layer at main path
  stage: integer, current stage label, used for generating layer names
  block: 'a','b'..., current block label, used for generating layer names
  strides: Strides for the second conv layer in the block.

Returns:
  A Keras model instance for the block.
"""
class ConvBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters, stage, block, strides=(2, 2), activation='relu'):
    self.activation = activation

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    super().__init__(name='stage-' + str(stage) + '_block-' + block)

    filters1, filters2, filters3 = filters
    bn_axis = -1

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1),
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2a')
    self.bn2a = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2a')
    self.act1  = tf.keras.layers.Activation(self.activation)

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size,
                                         strides=strides,
                                         padding='same',
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2b')
    self.bn2b = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2b')
    self.act2  = tf.keras.layers.Activation(self.activation)

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1),
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2c')
    self.bn2c = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2c')

    self.conv2s = tf.keras.layers.Conv2D(filters3, (1, 1),
                                         strides=strides,
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '1')
    self.bn2s = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '1')
    self.act3  = tf.keras.layers.Activation(self.activation)
    self.gauss1   = GaussianNoise(1)

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = self.act1(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = self.act2(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    shortcut = self.conv2s(input_tensor)
    shortcut = self.bn2s(shortcut, training=training)

    x = tf.keras.layers.add([x, shortcut])
    x = self.act3(x)
    x = self.gauss1(x)
    return x

class Discriminator(tf.keras.Model):
  def __init__(self, num_hidden, num_classes, activation='relu'):
    super().__init__(name='discriminator')
    self.hidden_layers = []
    for dim in num_hidden:
      self.hidden_layers.append(tf.keras.layers.Dense(dim, activation=activation))
    self.logits = tf.keras.layers.Dense(num_classes, activation=None)

  def call(self, x):
    for layer in self.hidden_layers:
      x = layer(x)
    x = self.logits(x)

    return x

"""Instantiates the ResNet50 architecture.

Args:
  num_classes: `int` number of classes for image classification.

Returns:
    A Keras model instance.
"""
class ResNet50(tf.keras.Model):
  def __init__(self, num_classes, num_features, activation='relu'):
    super().__init__(name='generator')
    bn_axis = -1
    self.activation = activation
    self.num_classes = num_classes

    self.conv1 = tf.keras.layers.Conv2D(32, (7, 7),
                                        strides=(2, 2),
                                        padding='valid',
                                        use_bias=False,
                                        kernel_initializer='he_normal',
                                        kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                        name='conv1')
    self.bn1 = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                  momentum=BATCH_NORM_DECAY,
                                                  epsilon=BATCH_NORM_EPSILON,
                                                  name='bn_conv1')
    self.act1 = tf.keras.layers.Activation(self.activation, name=self.activation+'1')
    self.max_pool1 = tf.keras.layers.MaxPooling2D((3, 3),
                                                  strides=(2, 2),
                                                  padding='same',
                                                  name='max_pool1')

    self.blocks = []
    self.blocks.append(ConvBlock(3, [32, 32, 128], strides=(1, 1), stage=2, block='a', activation=self.activation))
    self.blocks.append(IdentityBlock(3, [32, 32, 128], stage=2, block='b', activation=self.activation))

    self.blocks.append(ConvBlock(3, [64, 64, 256], stage=3, block='a', activation=self.activation))
    self.blocks.append(IdentityBlock(3, [64, 64, 256], stage=3, block='b', activation=self.activation))

    self.blocks.append(ConvBlock(3, [64, 64, 256], stage=4, block='a', activation=self.activation))
    self.blocks.append(IdentityBlock(3, [64, 64, 256], stage=4, block='b', activation=self.activation))

    self.avg_pool = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool')
    self.fc1 = tf.keras.layers.Dense(num_features,
                                     activation=self.activation,
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                     kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                     bias_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                     name='fc1')

    self.logits = tf.keras.layers.Dense(num_classes,
                                        activation=None,
                                        kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                        kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                        bias_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                        name='logits')

  def call(self, img_input, training=False):
    x = self.conv1(img_input)
    x = self.bn1(x, training=training)
    x = self.act1(x)
    x = self.max_pool1(x)

    for block in self.blocks:
      x = block(x)

    x = self.avg_pool(x)
    fc1 = self.fc1(x)
    logits = self.logits(fc1)

    return logits

def get_cross_entropy_loss(labels, logits):
  loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logits)
  return tf.reduce_mean(loss)

In [ ]:
cross_entropy_loss   = tf.keras.metrics.Mean(name='cross_entropy_loss')
temporal_test_acc    = tf.keras.metrics.CategoricalAccuracy(name='temporal_test_acc')
source_train_acc     = tf.keras.metrics.CategoricalAccuracy(name='source_train_acc')
source_test_acc      = tf.keras.metrics.CategoricalAccuracy(name='source_test_acc')
office_test_acc      = tf.keras.metrics.CategoricalAccuracy(name='office_test_acc')
server_train_acc     = tf.keras.metrics.CategoricalAccuracy(name='server_train_acc')
server_test_acc      = tf.keras.metrics.CategoricalAccuracy(name='server_test_acc')
conference_train_acc = tf.keras.metrics.CategoricalAccuracy(name='conference_train_acc')
conference_test_acc  = tf.keras.metrics.CategoricalAccuracy(name='conference_test_acc')

@tf.function
def test_step(images):
  return tf.nn.softmax(generator(images, training=False))

@tf.function
def train_gen_step(src_images, src_labels, ser_images, ser_labels, con_images, con_labels):
  with tf.GradientTape() as gen_tape:
    #Logits
    src_logits = generator(src_images, training=True)
    ser_logits = generator(ser_images, training=True)
    con_logits = generator(con_images, training=True)
    
    batch_cross_entropy_loss  = get_cross_entropy_loss(labels=tf.concat([src_labels, ser_labels, con_labels], 0),
                                                       logits=tf.concat([src_logits, ser_logits, con_logits], 0))

  gen_gradients = gen_tape.gradient(batch_cross_entropy_loss, generator.trainable_variables)
  gen_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))

  source_train_acc(src_labels, tf.nn.softmax(src_logits))
  server_train_acc(ser_labels, tf.nn.softmax(ser_logits))
  conference_train_acc(con_labels, tf.nn.softmax(con_logits))
  cross_entropy_loss(batch_cross_entropy_loss)

class lr_schedule():
  def __init__(self, init_lr=0.01, alpha=10, beta=0.75):
    self.init_lr = init_lr
    self.alpha = alpha
    self.beta = beta
    self.p = 0

  def set_p(self, p):
    self.p = p

  def __call__(self):
    return self.init_lr/((1+(self.alpha*self.p))**self.beta)

learning_rate  = lr_schedule(init_lr=init_lr)
generator      = ResNet50(num_classes, num_features, gen_activation)
gen_optimizer  = tf.keras.optimizers.Adam(learning_rate = learning_rate, beta_1 = 0.5)

summary_writer = tf.contrib.summary.create_file_writer('../logs/baselines/{}'.format(log_data), flush_millis=10000)
summary_writer.set_as_default()
global_step = tf.train.get_or_create_global_step()

for epoch in range(epochs):
  global_step.assign_add(1)

  for source_data, server_data, conf_data in zip(src_train_set, server_train_set, conf_train_set):
    train_gen_step(source_data[0], source_data[1], server_data[0], server_data[1], conf_data[0], conf_data[1])

  for data in time_test_set:
    temporal_test_acc(test_step(data[0]), data[1])

  for data in src_test_set:
    source_test_acc(test_step(data[0]), data[1])

  for data in office_test_set:
    office_test_acc(test_step(data[0]), data[1])

  for data in server_test_set:
    server_test_acc(test_step(data[0]), data[1])

  for data in conf_test_set:
    conference_test_acc(test_step(data[0]), data[1])

  with tf.contrib.summary.always_record_summaries():
    tf.contrib.summary.scalar("cross_entropy_loss", cross_entropy_loss.result())
    tf.contrib.summary.scalar("temporal_test_acc", temporal_test_acc.result())
    tf.contrib.summary.scalar("source_train_acc", source_train_acc.result())
    tf.contrib.summary.scalar("source_test_acc", source_test_acc.result())
    tf.contrib.summary.scalar("office_test_acc", office_test_acc.result())
    tf.contrib.summary.scalar("server_train_acc", server_train_acc.result())
    tf.contrib.summary.scalar("server_test_acc", server_test_acc.result())
    tf.contrib.summary.scalar("conference_train_acc", conference_train_acc.result())
    tf.contrib.summary.scalar("conference_test_acc", conference_test_acc.result())

  cross_entropy_loss.reset_states()
  temporal_test_acc.reset_states()
  source_train_acc.reset_states()
  source_test_acc.reset_states()
  office_test_acc.reset_states()
  server_train_acc.reset_states()
  server_test_acc.reset_states()
  conference_train_acc.reset_states()
  conference_test_acc.reset_states()